# Monte Carlo Prediction
---
> Monte Carlo
1. Policy Iteration
2. Value Iteration

* 불확실성을 예측하기 위해 sampling 사용 (trial - error)
* Value Function 을 사용하는 것은 model - free 에서 못쓰므로 여기서부터 Q Function 사용
* episode 의 모든 history 를 저장하므로 다음 상태에 대한 정보를 따로 알 필요없다. 이미 buffer 에 다 있으니까
* 또한 보상 에 해당하는 discounted Return 값이 target 이며 다음 상태의 가치 역할을 한다.

In [1]:
import gym
import numpy as np
import random
from gym.envs.registration import register

In [2]:
'''
환경셋팅 한 후에 환경을 추가등록한다.
'''

register(
    id='FrozenLake-v1',
    entry_point="gym.envs.toy_text:FrozenLakeEnv",
    kwargs={'map_name':'4x4','is_slippery':False})

In [3]:
'''
환경 생성
'''
env = gym.make('FrozenLake-v1')

In [10]:
q_table = np.zeros([env.action_space.n, env.observation_space.n], dtype = np.float16)
gamma = .9
epsilon = 1
episode = 0
max_episode = 1000
# episode 내용을 모두 저장한다.
history = np.zeros([1, 3])

state = env.reset()
action = env.action_space.sample()
step = 0

'''
compute discounted
'''
def discounted_reward(rewards):
    discounted = np.zeros_like(rewards)
    sum_tmp = 0
    
    for i in reversed(range(0, len(rewards))):
        sum_tmp = gamma * sum_tmp + rewards[i]
        discounted[i] = sum_tmp
    
    return discounted

'''
main
'''
while(episode < max_episode):
    
    step += 1
    state_next, reward, done, _ = env.step(action)
        
    if(random.random() > epsilon):
        action_next = np.argmax(q_table[ : , state_next])
    else:
        action_next = env.action_space.sample()
    
    state_old = state
    action_old = action
    
    state = state_next
    action = action_next
    
    # history 쌓기
    history = np.vstack((history, [state_old, action_old, reward]))
    
    # episode 끝나면 업데이트
    if(done):
        if(reward):
            if(epsilon > .1):
                epsilon = 1 / (episode/(max_episode/10) + 1)
            else:
                epsilon = .1
        else:
            history[-1, 2] = -1
            
        epr = discounted_reward(history[ : , 2])
        
        # M.C 업데이트
        for i in range(len(history)):
            s = int(history[i, 0])
            a = int(history[i, 1])
            r = epr[i]
            q_table[a, s] += gamma * (r - q_table[a, s])
            
        step = 0
        episode += 1
        env.reset()
        history = np.zeros([1, 3])

env.close()

In [11]:
q_table

array([[ 4.7827e-01,  5.0977e-01,  3.4241e-02, -1.2177e-01, -8.9014e-01,
        -6.0368e-04, -1.0000e+00, -1.9073e-04, -8.9014e-01,  6.4307e-01,
         7.2900e-01, -1.1986e-02, -1.0109e-03, -1.0000e+00,  8.1006e-01,
         5.3125e-01],
       [ 4.3823e-01, -1.0000e+00,  7.2900e-01, -1.0000e+00, -5.9180e-01,
         4.3604e-01,  8.1006e-01, -8.7402e-01, -1.0000e+00, -7.2900e-01,
         8.9990e-01, -2.0630e-02, -1.3342e-01,  7.7393e-01,  8.9990e-01,
        -4.8798e-02],
       [ 5.9033e-01,  6.5625e-01, -1.2878e-01, -1.6785e-01, -1.0000e+00,
        -4.1138e-01, -1.0000e+00, -8.1836e-01, -6.5723e-01, -7.3047e-01,
        -1.0000e+00, -8.1396e-01, -6.5771e-01,  8.9990e-01,  1.0000e+00,
        -9.2010e-03],
       [-8.0127e-01, -8.9014e-01,  5.9033e-01, -8.9014e-01,  5.2686e-01,
        -1.3863e-02,  6.5479e-01, -4.8853e-01, -8.8037e-01, -1.0000e+00,
         1.1871e-01, -7.6367e-01, -6.5771e-01,  6.8018e-01,  8.1006e-01,
         4.3042e-01]], dtype=float16)

In [12]:
s = env.reset()

step = 1
while(True):
    env.render()
    a = np.argmax(q_table[ : , s])
    s,r,d,_ = env.step(a)
    if(d):
        env.render()
        break
env.close()


SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
